In [74]:
from enum import IntEnum
from typing import List

import numpy as np

class MatrixAlgebra:
    @classmethod
    def egcd(cls, a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = cls.egcd(b % a, a)
            return (g, x - (b // a) * y, y)

    @classmethod
    def mod_inverse(cls, a, m):
        g, x, y = cls.egcd(a, m)
        if g != 1:
            raise ValueError(f'modular inverse does not exist: {a} as no inverse modulo {m}')
        else:
            return x % m
        
    @classmethod
    def minor_matrix(cls, m:np.matrix, i:int, j:int):
        return np.delete(np.delete(m, i, 0), j, 1)

    @classmethod
    def minor(cls, m:np.matrix, i:int, j:int):
        n = cls.minor_matrix(m, i, j)
        return cls.det(n)

    @classmethod
    def cofactor_matrix(cls, m:np.matrix) -> np.matrix:
        n = np.zeros_like(m)
        for (i0, i1), x in np.ndenumerate(m):
            x = np.delete(np.delete(m, i0, 0), i1, 1)
            c = 1 if (i0+i1) % 2 == 0 else -1
            n[i0, i1] = c*x
        return n

    @classmethod
    def adjugate(cls, m:np.matrix) -> np.matrix:
        return cls.cofactor_matrix(m).T
        
    @classmethod
    def det(cls, m:np.matrix):
        return round(np.linalg.det(m))
        
    @classmethod
    def det_inv_mod(cls, m:np.matrix, mod:int):
        det = np.linalg.det(m)
        det_inv = 1
        return det_inv

    @classmethod
    def inv(cls, m:np.matrix) -> np.matrix:
        det = cls.det(m)
        det_inv = 1/det
        adj = cls.adjugate(m)
        return det_inv * adj

    @classmethod
    def inv_mod(cls, m:np.matrix, mod:int) -> np.matrix:
        det = cls.det(m) % mod
        det_inv = cls.mod_inverse(det, mod)
        adj = cls.adjugate(m)
        n = (det_inv * adj) % mod 
        return n


LEXICON = IntEnum("Lexicon", [
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9",
    "A", "B", "C", "D", "E", "F", "G", "H", "I", "J",
    "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T",
    "U", "V", "W", "X", "Y", "Z", " ", "$", "%", "*",
    "+", "-", ".", "/", ":"
])

def encode(p: str) -> np.array:
    p = np.array([LEXICON[s].value - 1 for s in p])
    return p

def group(m: List[int], n: int) -> np.array:
    mat = m.reshape((-1, n), order='C')
    return mat
 
def decode(p):
    return "".join(map(lambda i: LEXICON(i + 1).name, p.flatten('F')))

def encrypt(m: np.array, k: np.array, mod: int):
    return np.matmul(k, m.T) % mod

def find_key(a, b, mod):
    for i in range(a.shape[0]):
        A = a[i:i+2, :]
        B = b[i:i+2, :]
        try:
            A_inv = MatrixAlgebra.inv_mod(A, mod)
            k = (np.matmul(A_inv, B) % mod).T
            k_inv = MatrixAlgebra.inv_mod(k, mod)
            return k, k_inv
        except ValueError as e:
            continue
    raise ValueError("Could not solve, try another key size.")


def do_crack(plaintext, cipher, mod):
    p_enc = group(encode(plaintext), 2)
    c_enc = group(encode(cipher), 2)
    k = find_key(p_enc, c_enc, mod)
    return k


def do_decipher(ciphertext, key, mod):
    n = key.shape[0]
    c_enc = group(encode(ciphertext), n)
    p_enc = encrypt(c_enc, key, mod)
    return decode(p_enc)
    
def do_cipher(plaintext, key, mod):
    n = key.shape[0]
    p_enc = group(encode(plaintext), n)
    c_enc = encrypt(p_enc, key, mod)
    return decode(c_enc)

In [75]:
cipher = "C3XJ%WZMOZ"
clear = "CODINGAME."
clear_me = "6-85OXC "
cipher_me = "HELLO WORLD."

In [76]:
k, k_inv = do_crack(clear, cipher, 45)
p_me = do_decipher(clear_me, k_inv, 45)
c_me = do_cipher(cipher_me, k, 45)
k, p_me, c_me

/tmp/ipykernel_223825/1747893275.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n[i0, i1] = c*x


(array([[6, 5],
        [7, 6]]),
 'BONJOUR.',
 '$N639O.8.0IS')